In [24]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)

In [25]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [26]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [27]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [33]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [34]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.160842  [    0/60000]
loss: 2.138976  [ 6400/60000]
loss: 2.088617  [12800/60000]
loss: 2.111855  [19200/60000]
loss: 2.076385  [25600/60000]
loss: 2.002399  [32000/60000]
loss: 2.033601  [38400/60000]
loss: 1.954914  [44800/60000]
loss: 1.950528  [51200/60000]
loss: 1.894025  [57600/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.886622 

Epoch 2
-------------------------------
loss: 1.919288  [    0/60000]
loss: 1.878617  [ 6400/60000]
loss: 1.770762  [12800/60000]
loss: 1.816167  [19200/60000]
loss: 1.723573  [25600/60000]
loss: 1.662073  [32000/60000]
loss: 1.684576  [38400/60000]
loss: 1.589574  [44800/60000]
loss: 1.613660  [51200/60000]
loss: 1.513072  [57600/60000]
Test Error: 
 Accuracy: 62.0%, Avg loss: 1.527100 

Epoch 3
-------------------------------
loss: 1.597811  [    0/60000]
loss: 1.551303  [ 6400/60000]
loss: 1.409665  [12800/60000]
loss: 1.483570  [19200/60000]
loss: 1.381862  [25600/60000]
loss: 1.363744  [32000/600